In [ ]:
import warnings
warnings.filterwarnings("ignore")
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
from tensorflow.keras import backend as K
import sklearn
import logging
import time
from tensorflow import keras

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
def data_generator(f_path):
    features = pd.read_excel(f_path)
    from sklearn import preprocessing
    from sklearn.preprocessing import MinMaxScaler
    scaler = preprocessing.StandardScaler().fit(features)
    print(scaler.mean_)
    print(scaler.scale_)
    input_features = scaler.transform(features)
    mean = scaler.mean_
    scale = scaler.scale_
    train_data = []
    train_label = []
    for i in range(480):   
        data1 = input_features[50*i:50*(i+1)]
        m = data1.tolist()
        train_data.append(m)
    print(len(test_label))
    print(train_data[0][0])
    return  np.array(train_data),mean,scale

In [ ]:
f_path = 'D:\date2\huizong3.xls'
train_datas,test_datas,mean,scale  = data_generator(f_path)

In [ ]:
train_datas = sklearn.utils.shuffle(train_datas)

In [ ]:

val_data = train_datas[:int(0.1*(len(train_datas)))]
# train_data1 = train_datas[:int(0.1*(len(train_datas)))]
train_data = train_datas[int(0.1*(len(train_datas))):]
b = tf.shape(train_data)
print(train_data.shape)

In [ ]:
# train_data = np.vstack((train_data1,train_data2))
# print(train_data.shape)

In [ ]:
x = train_data[:,:,:-1]
y = train_data[:,:,-1:]
x_val = val_data[:,:,:-1]
y_val = val_data[:,:,-1:]
print(x_val.shape)
x = tf.reshape(x,(b[0],50,6,1))
y = tf.reshape(y,(b[0],50,1,1))
x_val = tf.reshape(x_val,(47,50,6,1))
y_val = tf.reshape(y_val,(47,50,1,1))

In [ ]:
class Model(tf.keras.Model):
    def train_step(self, inputs):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = inputs

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
            loss = tf.reduce_mean(loss)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [ ]:
inputs = keras.Input(shape=(50,6,1))
cnn = keras.layers.Conv2D(50, 2, activation='relu')(inputs)
cnn = keras.layers.Conv2D(50, 2, activation='relu')(cnn)
cnn = keras.layers.Conv2D(50, 2, activation='relu')(cnn)
cnn = keras.layers.Conv2D(50, 2, activation='relu')(cnn)
print(cnn.shape)
a = tf.shape(cnn)
cnn = tf.reshape(cnn,(a[0],4600))
outputs = keras.layers.Dense(50)(cnn)
outputs = tf.reshape(outputs,(a[0],50,1))
model = Model(inputs, outputs)

In [ ]:
filepath = 'D:\data2\\CNN-best.h5'

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_mse', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)


model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss="mse", metrics=["mse"])
history = model.fit(x, y, epochs=300,validation_data=(x_val,y_val),batch_size=64, callbacks = [checkpoint])